In [131]:
from collections import Counter
import re

import numpy as np
from nltk.tokenize import sent_tokenize
from scipy.stats import entropy as kl_div
from math import log


In [2]:
data_path = '../../data/DUC/'
docs = {}

In [3]:
def get_docs(path):
    import os
    from bs4 import BeautifulSoup
    ret_dict = {}
    files = os.listdir(path)
    for file in files:
        file = path + file
        if os.path.isdir(file) or '.txt' in file:
            continue
        with open(path + file) as infile:
            soup = BeautifulSoup(infile, 'html.parser')
            ret_dict[soup.docno.text.strip()] = soup.find('text').text.strip()
    return ret_dict

In [4]:
docs = get_docs(data_path)

In [5]:
duc_data = [v for k, v in docs.items()]

In [6]:
regex = re.compile(r"(\W)")

In [141]:
def klsum(doc, L=250):
    
    class datatype(object):
        def __init__(self, text, reference=None):
            wc = Counter([s for s in re.split(regex, text) if s and s != ' ' and s != '\n'])
            if reference:
                self.wc = Counter({k: wc[k] for k, v in reference.wc.items()})
                if len(text) != 0:
                    self.pd = {k: v/sum(self.wc.values()) for k, v in self.wc.items()}
                else:
                    self.pd = {k: 0 for k, v in self.wc.items()}
            else:
                self.wc = wc
                if len(text) != 0:
                    self.pd = {k: v/sum(self.wc.values()) for k, v in self.wc.items()}
                else:
                    self.pd = {}
            self.text = text
            
        def __repr__(self):
            return self.text
            
    doc_dt = datatype(doc)
    sum_dt = datatype('', reference=doc_dt)
    
    def pick_best(doc_dt, sum_dt, sentences):
        for sentence in sentences:
            new_dt = datatype(sentence + ' ' + sum_dt.text, reference=doc_dt)
#             print(len(doc_dt.pd), len(new_dt.pd))
            print(kl_div(doc_dt.pd.values(), new_dt.pd.values()))
            
    sentences = sent_tokenize(doc)
    pick_best(doc_dt, sum_dt, sentences)

In [223]:
wc = lambda text: Counter([t for t in re.split(r"(\W)", text) if t and t != ' ' and t != '\n'])
pd = lambda wc: {k: v/sum(wc.values()) for k, v in wc.items()}

In [224]:
def klsum(document, summary, L):
    
    doc_sent = sent_tokenize(document)
    doc_wc = wc(document)
    doc_pd = pd(doc_wc)
    px = [p for p in doc_pd.values()]
    
    sum_sent = sent_tokenize(summary)
    if len(sum_sent) < L:
        if len(sum_sent) == 0:
            sum_wc = Counter({k: 0 for k in doc_wc.keys()})
            sum_pd = Counter({k: 0 for k in sum_wc.keys()})
        else:
            sum_wc = wc(summary)
            sum_pd = pd(wc)
            
        _min, min_id = 999, -1
        for index, sent in enumerate(doc_sent):
            new_wc = sum_wc + wc(sent)
            new_pd = pd(new_wc)
            qx = [new_pd[k] if k in new_pd else 0.001 for k in doc_pd.keys()]
            kl = kl_div(px, qx)
            if kl < _min:
                _min = kl
                min_id = index
        
        summary += doc_sent[min_id]
        document = " ".join(doc_sent[:index] + doc_sent[index+1:])
                
        return klsum(document, summary, L)
    else:
        return summary

In [225]:
klsum(docs['AP880217-0175'], '', 2)

AttributeError: 'function' object has no attribute 'items'